In [ ]:
import pandas as pd # Библиотека Pandas для работы с табличными данными
import numpy as np # библиотека Numpy для операций линейной алгебры и прочего
import matplotlib.pyplot as plt # библиотека Matplotlib для визуализации
import seaborn as sns # библиотека seaborn для визуализации
import numpy as np # библиотека Numpy для операций линейной алгебры и прочего

import plotly.graph_objects as go # Библиотека Plotly. Модуль "Graph Objects"
import plotly.express as px # Библиотека Plotly. Модуль "Express"

# предварительная обработка числовых признаков
from sklearn.preprocessing import MinMaxScaler# Импортируем нормализацию от scikit-learn
from sklearn.preprocessing import StandardScaler # Импортируем стандартизацию от scikit-learn
from sklearn.preprocessing import PowerTransformer  # Степенное преобразование от scikit-learn
# предварительная обработка категориальных признаков
from sklearn.preprocessing import OneHotEncoder# Импортируем One-Hot Encoding от scikit-learn
from sklearn.preprocessing import OrdinalEncoder# Импортируем Порядковое кодированиеот scikit-learn
from sklearn.preprocessing import LabelEncoder# Импортируем LabelEncoder от scikit-learn

from sklearn.pipeline import Pipeline # Pipeline.Не добавить, не убавить

from sklearn.compose import ColumnTransformer # т.н. преобразователь колонок

from sklearn.base import BaseEstimator, TransformerMixin # для создания собственных преобразователей / трансформеров данных

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!gdown --id 1wIY9kG9g2f6gwriz-RACYV7WyVAtgGWy

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1wIY9kG9g2f6gwriz-RACYV7WyVAtgGWy
To: /content/diabetes_prediction_dataset.csv
100% 3.81M/3.81M [00:00<00:00, 70.6MB/s]


In [ ]:
# Загружаем датасет людей с диабетом
df = pd.read_csv('/content/diabetes_prediction_dataset.csv')

In [ ]:
df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


# Preprocessing

In [ ]:
df.hypertension = df.hypertension.astype('object')
df.heart_disease = df.heart_disease.astype('object')
df.diabetes = df.diabetes.astype('object')

Разделяем числовые и категориальные фичи

In [ ]:
cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['gender', 'hypertension', 'heart_disease', 'smoking_history', 'diabetes'] 
 len =  5
numerical columns:	  ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level'] 
 len =  4


In [ ]:
class QuantileReplacer(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.05):
        self.threshold = threshold
        self.quantiles = {}

    def fit(self, X, y=None):
        for col in X.select_dtypes(include='number'):
            low_quantile = X[col].quantile(self.threshold)
            high_quantile = X[col].quantile(1 - self.threshold)
            self.quantiles[col] = (low_quantile, high_quantile)
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in X.select_dtypes(include='number'):
            low_quantile, high_quantile = self.quantiles[col]
            rare_mask = ((X[col] < low_quantile) | (X[col] > high_quantile))
            if rare_mask.any():
                rare_values = X_copy.loc[rare_mask, col]
                replace_value = np.mean([low_quantile, high_quantile])
                if rare_values.mean() > replace_value:
                    X_copy.loc[rare_mask, col] = high_quantile
                else:
                    X_copy.loc[rare_mask, col] = low_quantile
        return X_copy

In [ ]:
class RareGrouper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.05, other_value='Other'):
        self.threshold = threshold
        self.other_value = other_value
        self.freq_dict = {}

    def fit(self, X, y=None):
        for col in X.select_dtypes(include=['object']):
            freq = X[col].value_counts(normalize=True)
            self.freq_dict[col] = freq[freq >= self.threshold].index.tolist()
        return self

    def transform(self, X, y=None):
        X_copy = X.copy()
        for col in X.select_dtypes(include=['object']):
            X_copy[col] = X_copy[col].apply(lambda x: x if x in self.freq_dict[col] else self.other_value)
        return X_copy

Заменяем редкие значения и стандартизируем

In [ ]:
num_columns = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']
cat_columns = ['gender', 'hypertension', 'heart_disease', 'smoking_history']

In [ ]:
preprocessors_num = Pipeline([
    ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler()),
    ('power', PowerTransformer()),
])

In [ ]:
preprocessors_cat = Pipeline([
    ('encoder', OrdinalEncoder()),

])

In [ ]:
preprocessors_all = ColumnTransformer(transformers=[
    ('num_columns', preprocessors_num, num_columns),
    ('cat_columns', preprocessors_cat, cat_columns),
])

In [ ]:
preprocessors_all

ColumnTransformer(transformers=[('num_columns',
                                 Pipeline(steps=[('QuantReplace',
                                                  QuantileReplacer(threshold=0.01)),
                                                 ('scaler', StandardScaler()),
                                                 ('power',
                                                  PowerTransformer())]),
                                 ['age', 'bmi', 'HbA1c_level',
                                  'blood_glucose_level']),
                                ('cat_columns',
                                 Pipeline(steps=[('encoder',
                                                  OrdinalEncoder())]),
                                 ['gender', 'hypertension', 'heart_disease',
                                  'smoking_history'])])

In [ ]:
preprocessors_all.fit(df);

In [ ]:
# объединяем названия колонок в один список (важен порядок как в ColumnTransformer)
columns = np.hstack([num_columns,
                    cat_columns,
                    ])

In [ ]:
df_transformed = preprocessors_all.transform(df)
df_transformed = pd.DataFrame(df_transformed, columns=columns)

# Логистическая Регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression # Логистичекая регрессия от scikit-learn

from sklearn.model_selection import train_test_split#  функция разбиения на тренировочную и тестовую выборку
# в исполнении scikit-learn

from sklearn.model_selection import StratifiedKFold # при кросс-валидации разбиваем данные в пропорции целевой метки
from sklearn.model_selection import cross_validate # функция кросс-валидации от Scikit-learn

from sklearn.metrics import f1_score # f1-мера от Scikit-learn
from sklearn.metrics import classification_report # функция scikit-learn которая считает много метрик классификации

**Считываем данные, разбиваем на тестовую и тренировочную**

In [ ]:
# не забываем удалить целевую и категориальные переменные из признаков
X,y = df.drop(columns = ['diabetes', 'gender','hypertension','heart_disease','smoking_history']), df['diabetes']

## Приводим целевые метки к 0 и 1

Для этого воспользуемся объектом `LabelEncoder()` из модуля `preprocessing`

Применение преобразований уже стандартное для нас

* Создаем объект
* обучаем методом `.fit()`
* Смотрим что получилось

In [ ]:
Label = LabelEncoder()
Label.fit(y) # задаем столбец, который хотим преобразовать
Label.classes_ # в аттрибуте .classes_ хранится информация "какой класс как шифруется"

array([0, 1], dtype=object)

In [ ]:
target = Label.transform(y) # преобразуем и сохраняем в новую переменную

In [ ]:
target # здесь уже только 0 и 1

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# разбиваем на тренировочную и валидационную
X_train, X_val, y_train, y_val = train_test_split(X, target,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
X_train

,age,bmi,HbA1c_level,blood_glucose_level
76513,49.0,27.32,5.0,155
60406,64.0,27.32,3.5,145
27322,24.0,27.32,3.5,130
53699,55.0,27.32,6.5,159
65412,14.0,20.98,6.2,85
...,...,...,...,...
6265,49.0,32.98,5.7,80
54886,15.0,28.10,5.0,159
76820,42.0,26.14,5.8,85
860,37.0,24.96,6.2,158


# Оценим модель с использованием только числовых данных

In [ ]:
# Сначала обучаем на тренировочных данных
X_train_prep = preprocessors_num.fit_transform(X_train)
# потом на валидационной
X_val_prep = preprocessors_num.transform(X_val)



Стандартная уже для нас рутина

* Создаем объект класса
* "Обучаем" через `.fit`

In [ ]:
model = LogisticRegression(random_state = 42)

model.fit(X_train_prep, y_train);

## Подготовим несколько функций для анализа обученной модели

**вытаскивание коэффициентов из модели**

In [ ]:
def get_coefs (model):
    """Берем веса как атрибуты обученной модели.
    Входные переменные:
    ===========
    model: обученная модель
    """
    B0=model.intercept_[0]
    B=model.coef_[0]
    return B0, B

**написание модели**

In [ ]:
def print_model (B0,B,features_names):
    """Написание уравнения модели.
    Входные переменные:
    ===========
    B0: смещение (независимый коэффициент)
    weights: веса признаков
    features_names: список названий признаков
    """
    line='{:.3f}'.format(B0)
    sign=['+','-']
    for p,(fn,b) in enumerate(zip(features_names,B)):
        line=line+sign[int(0.5*(np.sign(b)-1))]+'{:.2f}*'.format(np.abs(b))+fn

    print('Решение')
    print(line)

**визуализации весов в виде столбчатых диаграмм**

In [ ]:
def vis_weigths(weights,features_names = None, width = 1200, height = 600):
    """Отрисовка весов.
    Входные переменные:
    ===========
    weights: веса признаков
    features_names: список названий признаков
    """
    numbers = np.arange(0,len(weights))                 # создаем массив от 0 до количество весов-1
    if features_names:
        tick_labels = np.hstack(['B0',features_names])
    else:
        tick_labels = ['B'+str(num) for num in numbers] # создаем "названия" весов
    fig = go.Figure()
    fig.add_trace(go.Bar(x=numbers[weights<0], y=weights[weights<0],
                    marker_color='red',
                    name='отрицательные веса'))
    fig.add_trace(go.Bar(x=numbers[weights>=0], y=weights[weights>=0],
                    marker_color='blue',
                    name='положительные веса'
                    ))

    fig.update_layout(
                title="Веса модели",
                width=width,
                height=height,
                template = "plotly_dark",
                xaxis = dict(
                                         tickmode = 'array',
                                         tickvals = numbers,
                                         ticktext = tick_labels,
                                         )
                )

    fig.show()

### Напишем уравнение модели

In [ ]:
B0, B = get_coefs(model)
features_names = list(num_columns)
print_model(B0,B,features_names)

Решение
-5.275+1.16*age+0.75*bmi+2.34*HbA1c_level+1.53*blood_glucose_level


### Визуализируем веса в виде столбчатых диаграмм

In [ ]:
Bs = np.hstack([B0,B])
vis_weigths(Bs,features_names)

## Оценка метрик

In [ ]:
def calculate_metric(model_pipe, X, y, metric = f1_score):
    """Расчет метрики.
    Параметры:
    ===========
    model_pipe: модель или pipeline
    X: признаки
    y: истинные значения
    metric: метрика (f1 - по умолчанию)
    """
    y_model = model_pipe.predict(X)
    return metric(y, y_model)

In [ ]:
print(f"f1 на тренировочной выборке: {calculate_metric(model, X_train_prep, y_train):.4f}")
print(f"f1 на валидационной выборке: {calculate_metric(model, X_val_prep, y_val):.4f}")

f1 на тренировочной выборке: 0.7212
f1 на валидационной выборке: 0.7008


## Кросс-валидация

In [ ]:
def cross_validation (X, y, model, scoring, cv_rule):
    """Расчет метрик на кросс-валидации.
    Параметры:
    ===========
    model: модель или pipeline
    X: признаки
    y: истинные значения
    scoring: словарь метрик
    cv_rule: правило кросс-валидации
    """
    scores = cross_validate(model,X, y,
                      scoring=scoring, cv=cv_rule )
    print('Ошибка на кросс-валидации')
    DF_score = pd.DataFrame(scores)
    display(DF_score)
    print('\n')
    print(DF_score.mean()[2:])

In [ ]:
scoring_clf = {'ACC': 'accuracy',
           'F1': 'f1',
           'Precision': 'precision',
           'Recall': 'recall'}

In [ ]:
cross_validation (X_train_prep, y_train,
                  model,
                  scoring_clf,
                  StratifiedKFold(n_splits=5, shuffle= True, random_state = 42))

Ошибка на кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,0.070121,0.028905,0.960071,0.724766,0.875149,0.618487
1,0.065151,0.028742,0.959071,0.720078,0.859977,0.619328
2,0.069418,0.029377,0.958714,0.713294,0.871515,0.603694
3,0.071742,0.029391,0.961714,0.735178,0.893157,0.624685
4,0.063468,0.029179,0.957929,0.709136,0.860911,0.602855




test_ACC          0.959500
test_F1           0.720490
test_Precision    0.872142
test_Recall       0.613810
dtype: float64


## Объединяем Pipeline Предобработки и Модель

In [ ]:
pipe_num = Pipeline([
    ('preprocessors', preprocessors_num),
    ('model', LogisticRegression(random_state = 42))
])

Чтобы "обратиться" к части Pipeline - достаточно помнить "имя"

Так же обучаем через `.fit()`

In [ ]:
pipe_num.fit(X_train, y_train)

Pipeline(steps=[('preprocessors',
                 Pipeline(steps=[('QuantReplace',
                                  QuantileReplacer(threshold=0.01)),
                                 ('scaler', StandardScaler()),
                                 ('power', PowerTransformer())])),
                ('model', LogisticRegression(random_state=42))])

In [ ]:
B0, B = get_coefs(pipe_num['model'])
features_names = list(num_columns)

print_model (B0,B,features_names)

Решение
-5.275+1.16*age+0.75*bmi+2.34*HbA1c_level+1.53*blood_glucose_level


In [ ]:
print(f"f1 на тренировочной выборке: {calculate_metric(pipe_num, X_train, y_train):.4f}")
print(f"f1 на валидационной выборке: {calculate_metric(pipe_num, X_val, y_val):.4f}")

f1 на тренировочной выборке: 0.7212
f1 на валидационной выборке: 0.7008


In [ ]:
cross_validation (X_train, y_train,
                  pipe_num,
                  scoring_clf,
                  StratifiedKFold(n_splits=5, shuffle= True, random_state = 42))

Ошибка на кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,0.457103,0.047886,0.960143,0.725394,0.875297,0.619328
1,0.467639,0.049481,0.959071,0.720078,0.859977,0.619328
2,0.462662,0.046961,0.958643,0.712655,0.871359,0.602855
3,0.464831,0.046111,0.961643,0.734553,0.893029,0.623846
4,0.475796,0.045410,0.957857,0.708498,0.860744,0.602015




test_ACC          0.959471
test_F1           0.720236
test_Precision    0.872081
test_Recall       0.613474
dtype: float64


# Оценим влияние категориальных признаков

In [ ]:
# возвращаем категориальные переменные
X,y = df.drop(columns = ['diabetes']), df['diabetes']

# разбиваем на тестовую и валидационную
X_train, X_val, y_train, y_val = train_test_split(X, target,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
pipe_all = Pipeline([
    ('preprocessors', preprocessors_all),
    ('model', LogisticRegression(random_state = 42))
    ])

In [ ]:
pipe_all.fit(X_train, y_train)

Pipeline(steps=[('preprocessors',
                 ColumnTransformer(transformers=[('num_columns',
                                                  Pipeline(steps=[('QuantReplace',
                                                                   QuantileReplacer(threshold=0.01)),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('power',
                                                                   PowerTransformer())]),
                                                  ['age', 'bmi', 'HbA1c_level',
                                                   'blood_glucose_level']),
                                                 ('cat_columns',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['gender', 'hypertension',
                                                   'heart_disease',
                                                   'smoking_history'])])),
                ('model', LogisticRegression(random_state=42))])

In [ ]:
# объединяем названия колонок в один список (важен порядок как в ColumnTransformer)
columns = np.hstack([num_columns,
                    cat_columns])

In [ ]:
B0, B = get_coefs(pipe_all['model'])
features_names = list(columns)

print_model (B0,B,features_names)

Решение
-5.698+1.02*age+0.71*bmi+2.33*HbA1c_level+1.53*blood_glucose_level+0.24*gender+0.73*hypertension+0.75*heart_disease+0.10*smoking_history


**Модификация функции для отрисовки весов**

In [ ]:
def vis_weigths_threshold(weights, feature_names, threshold, width = 1200, height = 600):
    """Отрисовка весов с фильтром.
    Входные переменные:
    ===========
    weights: веса признаков
    features_names: список названий признаков
    threshold: порог "значимости" коэффициентов
    """
    # фильтруем веса коэффициентов
    mask = np.abs(weights) >= threshold
    weights_filtered = weights[mask]
    feature_names_filtered = [name for name, m in zip(feature_names, mask[1:]) if m]

    # Рисуем столбчатую диаграмму
    vis_weigths(weights_filtered,feature_names_filtered, width = width, height = height)

In [ ]:
Bs = np.hstack([B0,B])
vis_weigths_threshold(Bs,columns, 0.5,  width = 1600)

In [ ]:
print(f"f1 на тренировочной выборке: {calculate_metric(pipe_all, X_train, y_train):.4f}")
print(f"f1 на валидационной выборке: {calculate_metric(pipe_all, X_val, y_val):.4f}")

f1 на тренировочной выборке: 0.7284
f1 на валидационной выборке: 0.7164


In [ ]:
cross_validation (X_train, y_train,
                  pipe_all,
                  scoring_clf,
                  StratifiedKFold(n_splits=5, shuffle= True, random_state = 42))

Ошибка на кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,0.596528,0.080357,0.959500,0.725424,0.856000,0.629412
1,0.606543,0.106659,0.960929,0.735621,0.865757,0.639496
2,0.621746,0.080237,0.959286,0.718657,0.871856,0.611251
3,0.611776,0.078211,0.961286,0.737403,0.871707,0.638959
4,0.594954,0.093695,0.958500,0.718371,0.849771,0.622166




test_ACC          0.959900
test_F1           0.727095
test_Precision    0.863018
test_Recall       0.628257
dtype: float64


Добавление категориальных признаков улучшило F1 нашей модели: `0.727` против `0.72`

# Улучшаем нашу модель (feature engineering)

In [ ]:
# Делим на группы по возрасту
bins = [0, 12, 18, 35, 60, 120]
labels = ['child', 'teenager', 'young_adult', 'adult', 'senior']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

# Делим по весу
bins_bmi = [0, 18.5, 24.9, 29.9, 100]
labels_bmi = ['underweight', 'normal_weight', 'overweight', 'obese']
df['bmi_category'] = pd.cut(df['bmi'], bins=bins_bmi, labels=labels_bmi, right=False)

# Группируем курящих
df['smoker'] = df['smoking_history'].apply(lambda x: 1 if x != 'never' and x != 'No Info' else 0)

# Делим по уровню глюкозы
bins_glucose = [0, 70, 100, 125, 1000]  # Define glucose level bins
labels_glucose = ['low', 'normal', 'elevated', 'high']
df['glucose_category'] = pd.cut(df['blood_glucose_level'], bins=bins_glucose, labels=labels_glucose, right=False)

In [ ]:
df.smoker = df.smoker.astype('object')
new_cat_columns = ['age_group', 'bmi_category', 'smoker', 'glucose_category']
cat_columns += new_cat_columns

In [ ]:
preprocessors_all = ColumnTransformer(transformers=[
    ('num_columns', preprocessors_num, num_columns),
    ('cat_columns', preprocessors_cat, cat_columns),
])

In [ ]:
preprocessors_all

ColumnTransformer(transformers=[('num_columns',
                                 Pipeline(steps=[('QuantReplace',
                                                  QuantileReplacer(threshold=0.01)),
                                                 ('scaler', StandardScaler()),
                                                 ('power',
                                                  PowerTransformer())]),
                                 ['age', 'bmi', 'HbA1c_level',
                                  'blood_glucose_level']),
                                ('cat_columns',
                                 Pipeline(steps=[('encoder',
                                                  OrdinalEncoder())]),
                                 ['gender', 'hypertension', 'heart_disease',
                                  'smoking_history', 'age_group',
                                  'bmi_category', 'smoker', 'glucose_category',
                                  'age_group', 'bmi_category', 'smoker',
                                  'glucose_category'])])

In [ ]:
preprocessors_all.fit(df);

In [ ]:
# объединяем названия колонок в один список (важен порядок как в ColumnTransformer)
columns = np.hstack([num_columns,
                    cat_columns])

In [ ]:
X,y = df.drop(columns = ['diabetes']), df['diabetes']
X_train, X_val, y_train, y_val = train_test_split(X, target,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
pipe_all = Pipeline([
    ('preprocessors', preprocessors_all),
    ('model', LogisticRegression(random_state = 42))
    ])

In [ ]:
pipe_all.fit(X_train, y_train)

Pipeline(steps=[('preprocessors',
                 ColumnTransformer(transformers=[('num_columns',
                                                  Pipeline(steps=[('QuantReplace',
                                                                   QuantileReplacer(threshold=0.01)),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('power',
                                                                   PowerTransformer())]),
                                                  ['age', 'bmi', 'HbA1c_level',
                                                   'blood_glucose_level']),
                                                 ('cat_columns',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['gender', 'hypertension',
                                                   'heart_disease',
                                                   'smoking_history',
                                                   'age_group', 'bmi_category',
                                                   'smoker', 'glucose_category',
                                                   'age_group', 'bmi_category',
                                                   'smoker',
                                                   'glucose_category'])])),
                ('model', LogisticRegression(random_state=42))])

In [ ]:
B0, B = get_coefs(pipe_all['model'])
features_names = list(columns)

print_model (B0,B,features_names)

Решение
-5.308+1.02*age+0.69*bmi+2.33*HbA1c_level+1.52*blood_glucose_level+0.22*gender+0.71*hypertension+0.74*heart_disease+0.08*smoking_history-0.03*age_group-0.06*bmi_category+0.13*smoker-0.09*glucose_category-0.03*age_group-0.06*bmi_category+0.13*smoker-0.09*glucose_category


**Модификация функции для отрисовки весов**

In [ ]:
def vis_weigths_threshold(weights, feature_names, threshold, width = 1200, height = 600):
    """Отрисовка весов с фильтром.
    Входные переменные:
    ===========
    weights: веса признаков
    features_names: список названий признаков
    threshold: порог "значимости" коэффициентов
    """
    # фильтруем веса коэффициентов
    mask = np.abs(weights) >= threshold
    weights_filtered = weights[mask]
    feature_names_filtered = [name for name, m in zip(feature_names, mask[1:]) if m]

    # Рисуем столбчатую диаграмму
    vis_weigths(weights_filtered,feature_names_filtered, width = width, height = height)

In [ ]:
Bs = np.hstack([B0,B])
vis_weigths_threshold(Bs,columns, 0.5,  width = 1600)

In [ ]:
print(f"f1 на тренировочной выборке: {calculate_metric(pipe_all, X_train, y_train):.4f}")
print(f"f1 на валидационной выборке: {calculate_metric(pipe_all, X_val, y_val):.4f}")

f1 на тренировочной выборке: 0.7299
f1 на валидационной выборке: 0.7153


In [ ]:
cross_validation (X_train, y_train,
                  pipe_all,
                  scoring_clf,
                  StratifiedKFold(n_splits=5, shuffle= True, random_state = 42))

Ошибка на кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,1.378892,0.231384,0.959929,0.727802,0.861079,0.630252
1,1.373361,0.229788,0.961357,0.738774,0.868331,0.642857
2,2.202841,0.225219,0.959429,0.719921,0.872162,0.612930
3,1.348030,0.194987,0.962071,0.742857,0.877574,0.643997
4,0.996758,0.134240,0.958071,0.714633,0.848730,0.617128




test_ACC          0.960171
test_F1           0.728797
test_Precision    0.865575
test_Recall       0.629433
dtype: float64


Добавление новых признаков улучшило F1 с `0.727` до почти `0.729`